# Data Cleaning

## Table of Contents
* [Fixing weather timestamp zone](#fix_timestamp)
* [Splitting the data into training and test sets](#splitting)
* [Missing values](#missing_values)
* [Further cleaning](#further)

In [4]:
# Libraries

%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import datetime as dt
import gc
import missingno as msno
import pandas_profiling
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import random

from src.functions import data_import as dimp
from src.functions import data_exploration as dexp
from src.functions import data_transformation as dtr

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pty

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Load train set
train_s4 = dimp.import_data('../../data/interim/site_4/train_s4_2.csv') 
train_s4['timestamp'] = pd.to_datetime(train_s4['timestamp'])
train_s4['year_built'] = pd.array(train_s4['year_built'], dtype=pd.Int16Dtype())
train_s4['floor_count'] = pd.array(train_s4['floor_count'], dtype=pd.Int16Dtype())
del train_s4['Unnamed: 0']

Memory usage of dataframe is 75.04 MB
Memory usage after optimization is: 19.71 MB
Decreased by 73.7%


### Fixing weather timestamp zone
<a id="fix_timestamp" />

In [3]:
# Convert weather timestamps in GMT to local time (GMT-8) (8 hours between maximum and minimum dayla temperature)
# We need to import raw data again and re-do the join
weather = dimp.import_data('../../data/raw/train/weather_train.csv')
train = dimp.import_data('../../data/raw/train/train.csv')
building_meta = dimp.import_data('../../data/raw/train/building_metadata.csv')

Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 2.65 MB
Decreased by 72.4%
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%


In [4]:
# Timestamps to datetime
train['timestamp'] = pd.to_datetime(train['timestamp']).astype('datetime64[ns]')
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

In [5]:
# Add -8 hours to weather timestamps
weather['timestamp'] += pd.DateOffset(hours=-8)

In [6]:
# Data join
df = pd.merge(
    pd.merge(building_meta, train, how='left', on='building_id'),
    weather,
    how='left',
    on=['site_id','timestamp']
)

df['year_built'] = pd.array(df['year_built'], dtype=pd.Int16Dtype())
df['floor_count'] = pd.array(df['floor_count'], dtype=pd.Int16Dtype())

# Rearranging columns
cols = [
    'site_id',
    'building_id', 
    'year_built', 
    'primary_use', 
    'floor_count', 
    'square_feet',
    'meter',  
    'timestamp',
    'air_temperature', 
    'cloud_coverage',
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_direction',
    'wind_speed',
    'meter_reading'
]

df = df[cols]

In [7]:
# Filter by site_id 4
df_s4 = dimp.get_data_by_site(df, 4)

In [8]:
# Let's check the result by visualizating the data
df_s4[df_s4.building_id == 565].set_index('timestamp')[
    [
        'air_temperature',
        'meter_reading'
    ]
].iplot(kind='scatter', filename='cufflinks/cf-simple-line')

### Splitting the data into training and test sets
<a id="splitting" />

We're using the two last months of each building observations for testing the models. The rest of rows will conform the training data set.

In [40]:
# select the last two months of each building
date_time_str = '2016-10-31 23:00:00'
date_time_cut = dt.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

# create test and training sets
test_s4 = df_s4[df_s4['timestamp'] > date_time_cut]
train_s4 = df_s4[df_s4['timestamp'] <= date_time_cut]

In [41]:
len(test_s4) + len(train_s4)

746746

In [42]:
test_s4.head()

,building_id,year_built,primary_use,floor_count,square_feet,meter,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading
6537747,565,1954,Education,2,15326,0,2016-11-01 00:00:00,16.703125,NaN,12.796875,NaN,1015.5,190.0,4.601562,8.25
6537748,565,1954,Education,2,15326,0,2016-11-01 01:00:00,16.093750,NaN,12.796875,NaN,1015.5,160.0,6.199219,8.50
6537749,565,1954,Education,2,15326,0,2016-11-01 02:00:00,15.000000,NaN,13.296875,NaN,1016.0,0.0,0.000000,8.00
6537750,565,1954,Education,2,15326,0,2016-11-01 03:00:00,15.601562,NaN,12.203125,NaN,1015.5,230.0,3.099609,8.25
6537751,565,1954,Education,2,15326,0,2016-11-01 04:00:00,15.601562,NaN,12.203125,NaN,1016.0,230.0,3.599609,8.25


In [43]:
train_s4.tail()

,building_id,year_built,primary_use,floor_count,square_feet,meter,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading
7275705,655,1964,Education,11,222434,0,2016-10-31 19:00:00,16.703125,NaN,11.703125,NaN,1014.5,170.0,7.699219,124.888000
7275706,655,1964,Education,11,222434,0,2016-10-31 20:00:00,16.093750,NaN,12.203125,NaN,1015.0,180.0,6.199219,122.206001
7275707,655,1964,Education,11,222434,0,2016-10-31 21:00:00,16.703125,NaN,13.296875,NaN,1015.0,220.0,5.699219,122.196999
7275708,655,1964,Education,11,222434,0,2016-10-31 22:00:00,16.703125,NaN,13.296875,NaN,1015.5,220.0,5.699219,120.597000
7275709,655,1964,Education,11,222434,0,2016-10-31 23:00:00,16.703125,NaN,12.796875,NaN,1015.0,200.0,4.601562,108.535004


In [44]:
# export training and test sets
train_s4.to_csv('../../data/interim/site_4/train_s4_2.csv')
test_s4.to_csv('../../data/interim/site_4/test_s4_2.csv')

### Missing values
<a id="missing_values"/>

In [66]:
# Çhecking missing values percentage
dexp.get_missing_percentage(
    train_s4.set_index(['building_id'])
)

year_built             1.24
primary_use            0.00
floor_count            0.00
square_feet            0.00
meter                  0.00
air_temperature        0.00
cloud_coverage        48.00
dew_temperature        0.00
precip_depth_1_hr     17.11
sea_level_pressure     0.00
wind_direction         0.00
wind_speed             0.00
meter_reading          0.00
dtype: float64

#### Interpolation for some weather attributes

Missing values in features in `dew_temperature`, `wind_direction` and `sea_level_pressure` will be filled by linear interpolation, concretly, by the method `time` of `interpolate()` pandas function.

In [ ]:
train_s4.set_index('timestamp', inplace=True)

In [65]:
# Input air_temperature, dew_temperature and wind_direction missing values by interpolation
train_s4['dew_temperature'].interpolate(method='time', inplace=True)
train_s4['wind_direction'].interpolate(method='time', inplace=True)
train_s4['sea_level_pressure'].interpolate(method='time', inplace=True)

#### KNN for `year_built`

Some building metadata features may depend on the building age (`year_built`). We can add a new building metada feature, `building_age`, to classify the buildings depending on the year they were built. We'll create three categories: 
* old: before 1940
* middle: between 1940 and 1980
* new: after 1980 ('new' seems to not sound good for 40 years old bildings, but here it's only a label).

This new feature will be used to input missing values in `year_built` by implementing the *KNN classification algorithm*: for every building with missing year built, KNN will classify it in one of the three age categories, which will allow to input the year built looking at the nearest neighbors year built value.

In [5]:
# Craete a copy of the training set and add the feature `building_age`.
train_cpy = train_s4.copy()
train_cpy = dtr.add_building_age_feature(train_cpy)

In [6]:
train_cpy.head()

,building_id,year_built,primary_use,floor_count,square_feet,meter,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading,building_age
0,565,1954,Education,2,15326,0,2016-01-01 00:00:00,4.398438,0.0,-2.199219,0.0,1021.0,0.0,0.000000,7.25,middle
1,565,1954,Education,2,15326,0,2016-01-01 01:00:00,4.398438,0.0,-4.398438,0.0,1020.5,20.0,2.099609,8.50,middle
2,565,1954,Education,2,15326,0,2016-01-01 02:00:00,4.398438,0.0,-6.699219,0.0,1021.0,20.0,2.099609,7.25,middle
3,565,1954,Education,2,15326,0,2016-01-01 03:00:00,4.398438,0.0,-7.800781,0.0,1020.5,30.0,2.599609,7.50,middle
4,565,1954,Education,2,15326,0,2016-01-01 04:00:00,5.000000,0.0,-9.398438,0.0,1020.5,0.0,0.000000,8.00,middle


### Further cleaning
<a id="further"/>

In [46]:
train_s4.isna().count()

building_id           614709
year_built            614709
primary_use           614709
floor_count           614709
square_feet           614709
meter                 614709
timestamp             614709
air_temperature       614709
cloud_coverage        614709
dew_temperature       614709
precip_depth_1_hr     614709
sea_level_pressure    614709
wind_direction        614709
wind_speed            614709
meter_reading         614709
dtype: int64

In [47]:
# Delete column `meter` as there's only one type of meter (electricity)
del train_s4['meter']

In [24]:
 train_s4.set_index('timestamp')[['air_temperature','dew_temperature']]

,air_temperature,dew_temperature
timestamp,,
2016-01-01 00:00:00,4.398438,-2.199219
2016-01-01 01:00:00,4.398438,-4.398438
2016-01-01 02:00:00,4.398438,-6.699219
2016-01-01 03:00:00,4.398438,-7.800781
2016-01-01 04:00:00,5.000000,-9.398438
...,...,...
2016-10-31 19:00:00,16.703125,11.703125
2016-10-31 20:00:00,16.093750,12.203125
2016-10-31 21:00:00,16.703125,13.296875
